In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np, pandas as pd
from fbprophet import Prophet
import math
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot

import warnings
warnings.filterwarnings("ignore")

In [9]:
#Data for years 2015-2018 are stored in a single file for each year
data2015 = pd.read_csv("mixed_cities/2015H1.csv", skiprows=4, parse_dates=["Date"], index_col="Date")
data2016 = pd.read_csv("mixed_cities/2016H1.csv", skiprows=4, parse_dates=["Date"], index_col="Date")
data2017 = pd.read_csv("mixed_cities/2017H1.csv", skiprows=4, parse_dates=["Date"], index_col="Date")
data2018 = pd.read_csv("mixed_cities/2018H1.csv", skiprows=4, parse_dates=["Date"], index_col="Date")

#Data for year 2019 are stored in 4 different files, collecting them all
data2019Q1 = pd.read_csv("mixed_cities/2019Q1.csv", skiprows=4, parse_dates=["Date"], index_col="Date")
data2019Q2 = pd.read_csv("mixed_cities/2019Q2.csv", skiprows=4, parse_dates=["Date"], index_col="Date")
data2019Q3 = pd.read_csv("mixed_cities/2019Q3.csv", skiprows=4, parse_dates=["Date"], index_col="Date")
data2019Q4 = pd.read_csv("mixed_cities/2019Q4.csv", skiprows=4, parse_dates=["Date"], index_col="Date")

#Data for year 2020 are stored in 4 different files, collecting them all
data2020Q1 = pd.read_csv("mixed_cities/2020Q1.csv", skiprows=4, parse_dates=["Date"], index_col="Date")
data2020Q2 = pd.read_csv("mixed_cities/2020Q2.csv", skiprows=4, parse_dates=["Date"], index_col="Date")
data2020Q3 = pd.read_csv("mixed_cities/2020Q3.csv", skiprows=4, parse_dates=["Date"], index_col="Date")
data2020Q4 = pd.read_csv("mixed_cities/2020Q4.csv", skiprows=4, parse_dates=["Date"], index_col="Date")

#Data for year 2021 are stored in 4 different files, collecting them all
data2021Q1 = pd.read_csv("mixed_cities/2021Q1.csv", parse_dates=["Date"], index_col="Date")
data2021Q2 = pd.read_csv("mixed_cities/2021Q2.csv", parse_dates=["Date"], index_col="Date")
data2021Q3 = pd.read_csv("mixed_cities/2021Q3.csv", parse_dates=["Date"], index_col="Date")
data2021Q4 = pd.read_csv("mixed_cities/2021Q4.csv", parse_dates=["Date"], index_col="Date")

#Data for year 2022
data2022 = pd.read_csv("mixed_cities/2022.csv", parse_dates=["Date"], index_col="Date")

#concatenate the different dataframes collected for years 2019-2021 and concatenate them into one dataframe per year
all2019 = [data2019Q1, data2019Q2, data2019Q3, data2019Q4]
all2020 = [data2020Q1, data2020Q2, data2020Q3, data2020Q4]
all2021 = [data2021Q1, data2021Q2, data2021Q3, data2021Q4]

data2019 = pd.concat(all2019)
data2020 = pd.concat(all2020)
data2021 = pd.concat(all2021)


#The column 'Specie' holds the pollutants and weather conditions that affect air pollution
#To manage each pollutant as a separate time series, I made every unique value in 'Specie' into a separate column
#If there are multiple rows of the same [date, country, city] and the same pollutant I store the mean of the values

cleaned2015 = data2015.pivot_table(index=['Date', 'Country', 'City'], 
                     columns='Specie', 
                     values='median', 
                     fill_value='',
                     aggfunc=np.mean).rename_axis(None, axis=1).reset_index()
cleaned2016 = data2016.pivot_table(index=['Date', 'Country', 'City'], 
                     columns='Specie', 
                     values='median', 
                     fill_value='',
                     aggfunc=np.mean).rename_axis(None, axis=1).reset_index()
cleaned2017 = data2017.pivot_table(index=['Date', 'Country', 'City'], 
                     columns='Specie', 
                     values='median', 
                     fill_value='',
                     aggfunc=np.mean).rename_axis(None, axis=1).reset_index()
cleaned2018 = data2018.pivot_table(index=['Date', 'Country', 'City'], 
                     columns='Specie', 
                     values='median', 
                     fill_value='',
                     aggfunc=np.mean).rename_axis(None, axis=1).reset_index()
cleaned2019 = data2019.pivot_table(index=['Date', 'Country', 'City'], 
                     columns='Specie', 
                     values='median', 
                     fill_value='',
                     aggfunc=np.mean).rename_axis(None, axis=1).reset_index()
cleaned2020 = data2020.pivot_table(index=['Date', 'Country', 'City'], 
                     columns='Specie', 
                     values='median', 
                     fill_value='',
                     aggfunc=np.mean).rename_axis(None, axis=1).reset_index()
cleaned2021 = data2021.pivot_table(index=['Date', 'Country', 'City'], 
                     columns='Specie', 
                     values='median', 
                     fill_value='',
                     aggfunc=np.mean).rename_axis(None, axis=1).reset_index()
cleaned2022 = data2022.pivot_table(index=['Date', 'Country', 'City'], 
                     columns='Specie', 
                     values='median', 
                     fill_value='',
                     aggfunc=np.mean).rename_axis(None, axis=1).reset_index()

In [3]:
cleaned2022

,Date,Country,City,aqi,co,dew,humidity,mepaqi,neph,no2,...,pm10,pm25,precipitation,pressure,so2,temperature,uvi,wd,wind-gust,wind-speed
0,2021-12-27,AE,Abu Dhabi,,0.1,16,68,,,5.4,...,49,21,,1018,6.4,22.2,,,2.4,4.1
1,2021-12-27,AE,Dubai,,,15,57,,,,...,,59,,1017,,24,,,,4.1
2,2021-12-27,AR,Buenos Aires,,0.1,19.3,71.3,,,11.5,...,20,21,,1008.9,0.6,24.8,,112.5,4.6,2.5
3,2021-12-27,AT,Graz,,0.1,-1.1,33,,,11,...,16,,99,1009,0.6,0.5,,,3.8,1.2
4,2021-12-27,AT,Innsbruck,,0.1,-1.7,75.7,,,17,...,18,,,1007,0.6,2.9,,,10,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47945,2022-03-19,ZA,Durban,,,,84,,,,...,,,,1022.2,,21.4,,,5.6,3
47946,2022-03-19,ZA,Johannesburg,,,,76.5,,,,...,,,,,,,,,1.6,1
47947,2022-03-19,ZA,Pretoria,,,13,,,,,...,,,,,,,,,1.5,
47948,2022-03-19,ZA,Vereeniging,,,,79.2,,,,...,,,,,,,,,1.7,


In [4]:
#DATA PREPARATION FOR UNIVARIATE PROPHET MODEL

#GATHER ATHENS DATA FROM ALL AVAILABLE YEARS
Athens_Data = pd.concat([cleaned2015[cleaned2015['City']=='Athens'], cleaned2016[cleaned2016['City']=='Athens'],
                     cleaned2017[cleaned2017['City']=='Athens'], cleaned2018[cleaned2018['City']=='Athens'], 
                     cleaned2019[cleaned2019['City']=='Athens'], cleaned2020[cleaned2020['City']=='Athens'],
                     cleaned2021[cleaned2021['City']=='Athens'], cleaned2022[cleaned2022['City']=='Athens']])
#KEEP ONLY THE POLLUTANTS COLUMNS
cols=['pm25', 'pm10', 'no2', 'o3', 'so2', 'Date']

#MAKE VALUES NUMERIC
Athens_Data_uni = Athens_Data.loc[:, Athens_Data.columns.intersection(cols)]
Athens_Data_uni[['pm25', 'pm10', 'no2', 'o3', 'so2']] = Athens_Data_uni[['pm25', 'pm10', 'no2', 'o3', 'so2']].apply(pd.to_numeric, errors='coerce')

#FILL NAN VALUES
Athens_Data_uni = Athens_Data_uni.iloc[1: , :]

#SOME VALUES ARE EMPTY, BUT NOT RECOGNIZED AS NAN. REPLACE EMPTY VALUES WITH NAN TO FILL THEM LATER
Athens_Data_uni = Athens_Data_uni.replace(r'^\s*$', np.nan, regex=True)
Athens_Data_uni = Athens_Data_uni.fillna(method='ffill')
Athens_Data_uni = Athens_Data_uni.fillna(method='bfill')

#REMOVE DUPLICATE VALUES
Athens_Data_uni = Athens_Data_uni[~Athens_Data_uni.index.duplicated(keep='first')]

#Fb prophet model predicts univariate time series. It requires that the Date index column is named 'ds'
#and the values column is named 'y'
Athens_Data_uni.rename(columns = {'Date':'ds'}, inplace = True)
Athens_Data_uni['ds'] = pd.DatetimeIndex(Athens_Data_uni['ds'])

In [5]:
Athens_Data_uni

,ds,no2,o3,pm10,pm25,so2
176072,2019-11-07,17.4,27.7,48.0,54.0,1.1
176663,2019-11-08,13.3,27.7,31.0,54.0,1.6
177253,2019-11-09,13.8,26.9,17.0,41.0,1.6
177845,2019-11-10,8.3,30.1,12.0,33.0,1.6
178437,2019-11-11,11.5,22.8,18.0,36.0,1.6
...,...,...,...,...,...,...
45324,2022-03-15,11.9,31.3,24.0,61.0,4.6
45902,2022-03-16,13.8,31.7,30.0,74.0,5.1
46480,2022-03-17,11.0,33.8,32.0,70.0,3.6
47057,2022-03-18,7.4,36.2,22.0,53.0,2.1


In [6]:
def fbprophet_predict(time_series, col):
    
    #CREATE MODEL
    model = Prophet()
    
    #SPLIT TO TRAIN AND TEST PORTIONS
    train_size = 0.8
    train = time_series[:int(train_size*(len(time_series)))]
    test = time_series[int(train_size*(len(time_series))):]
    
    #FIT TRAIN DATA TO MODEL
    model.fit(train)
    
    #PREDICT THE TEST PORTION. THE PARAMETER IN THE PREDICT FUNCTION IS THE TIME INTERVAL TO PREDICT
    forecast = model.predict(test[['ds']])
    
    #FORECAST PREVIEW. 'ds' IS THE DAILY INDEX, 'yhat' IS THE FORECAST, 'yhat_lower' and 'yhat_upper' ARE THE
    #LOWER AND UPPER BOUND OF THE FORECASTED VALUE RESPECTIVELY.
    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
    print(test.reset_index(drop=True).head())
    
    #COMPUTE RMSE OF THE FORECASTED AND ACTUAL VALUE
    rmse = math.sqrt(mean_squared_error(test[['y']], forecast[['yhat']]))
    
    print("FB Prophet RMSE for Athens[" + col + "]: " + str(rmse) + "\n")

    return test, forecast, rmse

In [7]:
#FB PROPHET REQUIRES THE DATAFRAME TO HAVE TWO COLUMNS. 'ds' AND 'y', 'ds' IS THE TIME INDEX AND 'y' IS THE
#VALUE OF THE POLLUTANT

#RENAME COLUMNS TO FIT TO THE PROPHET MODEL
Athens_o3 = Athens_Data_uni[['ds', 'o3']]
Athens_o3.rename(columns = {'o3':'y'}, inplace = True)

Athens_no2 = Athens_Data_uni[['ds', 'no2']]
Athens_no2.rename(columns = {'no2':'y'}, inplace = True)

Athens_so2 = Athens_Data_uni[['ds', 'so2']]
Athens_so2.rename(columns = {'so2':'y'}, inplace = True)

Athens_pm10 = Athens_Data_uni[['ds', 'pm10']]
Athens_pm10.rename(columns = {'pm10':'y'}, inplace = True)

Athens_pm25 = Athens_Data_uni[['ds', 'pm25']]
Athens_pm25.rename(columns = {'pm25':'y'}, inplace = True)

datasets = [Athens_o3, Athens_no2, Athens_so2, Athens_pm10, Athens_pm25]

In [8]:
pollutants = ['o3', 'no2', 'so2', 'pm10', 'pm25']

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict(data, pollutants[i])
    i = i+1

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds       yhat  yhat_lower  yhat_upper
0 2021-09-09  46.104857   38.231399   53.813977
1 2021-09-10  45.536028   37.641998   52.843560
2 2021-09-11  46.186653   38.449265   53.728978
3 2021-09-12  47.338121   39.703093   55.257735
4 2021-09-13  44.754882   37.112847   52.580131
          ds     y
0 2021-09-09  30.5
1 2021-09-10  31.7
2 2021-09-11  35.8
3 2021-09-12  46.8
4 2021-09-13  45.9
FB Prophet RMSE for Athens[o3]: 31.837200018523635



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-09  8.535939    2.772386   14.239776
1 2021-09-10  9.356604    3.659754   15.220752
2 2021-09-11  8.480783    2.928853   14.283282
3 2021-09-12  6.259384    0.287052   11.626397
4 2021-09-13  8.897427    3.115820   14.350448
          ds     y
0 2021-09-09   6.4
1 2021-09-10   9.6
2 2021-09-11  12.4
3 2021-09-12  13.8
4 2021-09-13  17.0
FB Prophet RMSE for Athens[no2]: 6.364385806631491



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-09  4.894622    3.104912    6.598856
1 2021-09-10  5.025540    3.378004    6.842946
2 2021-09-11  4.864910    3.174271    6.647045
3 2021-09-12  4.703262    3.020251    6.361873
4 2021-09-13  4.828877    3.206174    6.592065
          ds    y
0 2021-09-09  4.1
1 2021-09-10  4.1
2 2021-09-11  2.6
3 2021-09-12  3.6
4 2021-09-13  3.6
FB Prophet RMSE for Athens[so2]: 2.737309435626895



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds       yhat  yhat_lower  yhat_upper
0 2021-09-09  26.779960   13.874796   39.093302
1 2021-09-10  25.894284   13.736543   37.641106
2 2021-09-11  27.773824   14.486700   40.154664
3 2021-09-12  27.612442   16.421857   40.153711
4 2021-09-13  27.013903   14.526761   39.376572
          ds     y
0 2021-09-09  12.0
1 2021-09-10  18.0
2 2021-09-11  22.0
3 2021-09-12  30.0
4 2021-09-13  32.0
FB Prophet RMSE for Athens[pm10]: 10.770589156232493

          ds       yhat  yhat_lower  yhat_upper
0 2021-09-09  55.528697   38.675556   72.040574
1 2021-09-10  54.918237   37.161790   72.006690
2 2021-09-11  58.259074   42.083220   75.038851
3 2021-09-12  59.251441   41.525289   75.515922
4 2021-09-13  56.374108   39.950736   73.324980
          ds     y
0 2021-09-09  30.0
1 2021-09-10  50.0
2 2021-09-11  53.0
3 2021-09-12  57.0
4 2021-09-13  61.0
FB Prophet RMSE for Athens[pm25]: 13.552166255491628

